### Import modules

In [1]:
import tensorflow as tf

# from https://medium.com/ibm-data-ai/memory-hygiene-with-tensorflow-during-model-training-and-deployment-for-inference-45cf49a15688
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(str(gpu))
        tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])


import tensorflow.keras as keras
from keras import layers
import numpy as np
from dataset import *

2023-01-01 22:46:04.016470: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-01 22:46:04.508815: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-01 22:46:05.918972: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-01 22:46:05.919134: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


#### GPU config

### Create dataset from test audio file


In [2]:
xf, X = create_dataset('audio/training_audio.wav', 2, 0.5)
print(X.shape)

STRIDE IS 0.007661108887999645
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.64688427299703
helo 255.

## Define model

In [3]:
inputs = keras.Input((1024, 256, 1))

# Encoder
conv = layers.Conv2D(2, (3, 3), (1, 1), padding='same', activation='relu')(inputs)
pool = layers.MaxPool2D((2, 2), strides=2)(conv)
conv = layers.Conv2D(4, (3, 3), (1, 1), padding='same', activation='relu')(pool)
pool = layers.MaxPool2D((2, 2), strides=2)(conv)
conv = layers.Conv2D(8, (3, 3), (1, 1), padding='same', activation='relu')(pool)
pool = layers.MaxPool2D((2, 2), strides=2)(conv)
conv = layers.Conv2D(16, (3, 3), (1, 1), padding='same', activation='relu')(pool)
pool = layers.MaxPool2D((2, 2), strides=2)(conv)
conv = layers.Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu')(pool)
pool = layers.MaxPool2D((2, 2), strides=2)(conv)
conv = layers.Conv2D(64, (3, 3), (1, 1), padding='same', activation='relu')(pool)
pool = layers.MaxPool2D((2, 2), strides=2)(conv)
conv = layers.Conv2D(128, (3, 3), (1, 1), padding='same', activation='relu')(pool)
pool = layers.MaxPool2D((2, 2), strides=2)(conv)


flatten = layers.Flatten()(pool)
dense = layers.Dense(512)(flatten)

# Decoder
dense2 = layers.Dense(flatten.shape[1])(dense)
reshaped = layers.Reshape(pool.shape[1:])(dense2)

filters = 128
deconv = reshaped
while (filters > 2):
    print(filters)
    depool = layers.Conv2DTranspose(filters, (2, 2), strides=2)(deconv)
    deconv = layers.Conv2DTranspose(1, (3, 3), strides=1, padding='same', activation='relu')(depool)
    filters /= 2

depool = layers.Conv2DTranspose(filters, (2, 2), strides=2)(deconv)
deconv = layers.Conv2DTranspose(1, (3, 3), strides=1, padding='same', activation='sigmoid')(depool)

print(deconv.shape)

outputs = deconv

model = keras.Model(inputs=inputs, outputs=outputs, name="autoencoder")

2023-01-01 22:46:38.881112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-01 22:46:38.884015: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-01 22:46:38.884984: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-01 22:46:38.885803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

128
64.0
32.0
16.0
8.0
4.0
(None, 1024, 256, 1)


In [4]:
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 256, 1)]    0         
                                                                 
 conv2d (Conv2D)             (None, 1024, 256, 2)      20        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 512, 128, 2)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 512, 128, 4)       76        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 256, 64, 4)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 64, 8)        

In [6]:
X_fit = tf.expand_dims(X, -1)
X_fit = (X_fit - np.min(X_fit))/(np.max(X_fit) - np.min(X_fit))

print("==================")
print(X_fit.shape)
print("==================")

model.fit(X_fit, X_fit,
        epochs=500,
        shuffle=True
)


(116, 1024, 256, 1)
Epoch 1/500


2023-01-01 22:47:46.332768: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8700
2023-01-01 22:47:48.387898: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-01 22:47:48.877119: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


4/4 [==============================] - 9s 323ms/step - loss: 0.6918
Epoch 2/500
4/4 [==============================] - 0s 87ms/step - loss: 0.6877
Epoch 3/500
4/4 [==============================] - 0s 81ms/step - loss: 0.6831
Epoch 4/500
4/4 [==============================] - 0s 79ms/step - loss: 0.6780
Epoch 5/500
4/4 [==============================] - 0s 82ms/step - loss: 0.6722
Epoch 6/500
4/4 [==============================] - 0s 78ms/step - loss: 0.6654
Epoch 7/500
4/4 [==============================] - 0s 81ms/step - loss: 0.6575
Epoch 8/500
4/4 [==============================] - 0s 76ms/step - loss: 0.6479
Epoch 9/500
4/4 [==============================] - 0s 81ms/step - loss: 0.6358
Epoch 10/500
4/4 [==============================] - 0s 80ms/step - loss: 0.6203
Epoch 11/500
4/4 [==============================] - 0s 74ms/step - loss: 0.5995
Epoch 12/500
4/4 [==============================] - 0s 81ms/step - loss: 0.5711
Epoch 13/500
4/4 [==============================] - 0s 77ms/

### COMPARE SPECTROGRAMS IN IMAGES


In [7]:
import matplotlib.pyplot as plt

In [8]:
x_example = X[0]

plt.imsave("INPUT_EXAMPLE.png", x_example)
x_example = np.reshape(x_example, (1, x_example.shape[0], x_example.shape[1], 1))
print(x_example.shape)
prediction = model.predict(x_example)
plt.imsave("OUTPUT_EXAMPLE.png", prediction[0, :, :, 0])

(1, 1024, 256, 1)
1/1 [==============================] - 2s 2s/step


In [14]:
print(prediction)

[[[[0.52592367]
   [0.7796018 ]
   [0.7836387 ]
   ...
   [0.7794041 ]
   [0.77330756]
   [0.62095875]]

  [[0.81368977]
   [1.1694833 ]
   [1.173792  ]
   ...
   [1.1690413 ]
   [1.1652687 ]
   [0.921515  ]]

  [[0.81778747]
   [1.1748464 ]
   [1.1750174 ]
   ...
   [1.1728822 ]
   [1.1707551 ]
   [0.9278976 ]]

  ...

  [[0.82186157]
   [1.1761847 ]
   [1.1761847 ]
   ...
   [1.1761261 ]
   [1.175649  ]
   [0.9287098 ]]

  [[0.82186157]
   [1.1761847 ]
   [1.1761847 ]
   ...
   [1.1758169 ]
   [1.1749599 ]
   [0.9286688 ]]

  [[0.5448024 ]
   [0.76198274]
   [0.76198274]
   ...
   [0.76158   ]
   [0.7609545 ]
   [0.58741355]]]]
